In [1]:
# Move software to working disk
!rm  -r software
!scp -r /kaggle/input/graphnet-and-dependencies/software .

# Install dependencies
!pip install /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/working/software/dependencies/torch_geometric-2.0.4.tar.gz

!cd software/graphnet;pip install --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

rm: cannot remove 'software': No such file or directory
Processing ./software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
Processing ./software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
Processing ./software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
Processing ./software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
Processing ./software/dependencies/torch_geometric-2.0.4.tar.gz
  Preparing metadata (setup.py) ... - done
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=1995e0391364c639745f7fce866d5f32e15007d9577704d70438ee3e1fb138a1
  Stored in directory: /root/.cache/pip/wheels/c0/33/a3/07aa146f758cd91ebee36268011873ae31c2cfc59dec089e04
Successfully built torch-geometric
Looking in links: /kaggle/working/software

In [2]:
# Install GraphNeT
import sys
#sys.path.append('/kaggle/input/graphnet/graphnet/src')
sys.path.append('/kaggle/working/software/graphnet/src')
import graphnet

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyarrow 

DATA_PATH = '/kaggle/input/icecube-neutrinos-in-deep-ice/'
SENSORS = DATA_PATH + 'sensor_geometry.csv'
TRANSPERANCY = '/kaggle/input/icecube-additional/ice_transperancy.txt'

import sys
sys.path.append('/kaggle/input/icecube-utils/')
from prepare_sensors import prepare_sensors
from ice_transparency import ice_transparency
sensor_df = prepare_sensors(SENSORS)
f_scattering, f_absorption = ice_transparency(TRANSPERANCY)

In [4]:
META_PATH = '/kaggle/input/batched-metadata/'
def get_metadata_pd(batch, write=False):
    if batch < 661:
        return pd.read_parquet(META_PATH + f'train_meta_batches/batch_{batch}.parquet', 
                        engine="pyarrow", use_threads=True)
    elif batch == 661:
        return pd.read_parquet(META_PATH + f'test_meta_batches/batch_{batch}.parquet', 
                        engine="pyarrow", use_threads=True)
    

In [5]:
import torch
from torch_geometric.data import Data
from graphnet.training.labels import Label

class Direction(Label):
    """Class for producing my label."""
    def __init__(self):
        """Construct `MyCustomLabel`."""
        # Base class constructor
        super().__init__(key="direction")

    def __call__(self, graph: Data) -> torch.tensor:
        """Compute label for `graph`."""
        zenith = graph.y[0]
        azimuth = graph.y[1] # assuming y is a pandas dataframe
               
        dir_x = (torch.cos(azimuth) * torch.sin(zenith)).reshape(1)
        dir_y = (torch.sin(azimuth) * torch.sin(zenith)).reshape(1)
        dir_z = torch.cos(zenith).reshape(1)
        direction = torch.cat([dir_x, dir_y, dir_z], dim=0)
        return direction


/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [6]:
import torch
from torch_geometric.nn import knn_graph
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from typing import (
    cast,
    Any,
    Callable,
    Dict,
    List,
    Optional,
    Tuple,
    Union,
    Iterable,
)

class IceCubeDataset(Dataset):
    def __init__(self, event_ids, batch_id, PATH_TO_BATCH_FILES, 
                 f_scattering, f_absorption, sensor_df, y, x_features, y_features,
                 pulse_limit=300, include_auxiliary=True, construct_graph=False,
                 transform = None, pre_transform=None, pre_filter=None):
        super().__init__(transform, pre_transform, pre_filter)
        self.event_ids = event_ids
        self.batch_df = pd.read_parquet(PATH_TO_BATCH_FILES + f"batch_{batch_id}.parquet")
        self.sensor_df = sensor_df
        self.pulse_limit = pulse_limit
        self.f_scattering = f_scattering
        self.f_absorption = f_absorption
        self.y = y
        self.x_features = x_features
        if include_auxiliary == False and 'auxiliary' in self.x_features:
            self.x_features.remove('auxiliary')
        self.include_auxiliary = include_auxiliary
        self.y_features = y_features
        self.construct_graph = construct_graph
        self._label_fns = dict()
        
        
        # weird scaling...really don't get any of the scaling stuff
        self.batch_df["time"] = (self.batch_df["time"] - 1.0e04) / 3.0e4
        self.batch_df["charge"] = np.log10(self.batch_df["charge"]) / 3.0
        self.batch_df["auxiliary"] = self.batch_df["auxiliary"].astype(int) - 0.5
       
    def len(self):
        return len(self.event_ids)
    
    def get_dir_vector(self, azimuth, zenith):
        dir_x = np.cos(azimuth) * np.sin(zenith)
        dir_y = np.sin(azimuth) * np.sin(zenith)
        dir_z = np.cos(zenith)
        directions = pd.Series({'direction_x':dir_x, 'direction_y':dir_y, 'direction_z':dir_z})
        return directions
    
    def add_label(
        self, fn: Callable[[Data], Any], key: Optional[str] = None
    ) -> None:
        """Add custom graph label define using function `fn`."""
        if isinstance(fn, Label):
            key = fn.key
        assert isinstance(
            key, str
        ), "Please specify a key for the custom label to be added."
        assert (
            key not in self._label_fns
        ), f"A custom label {key} has already been defined."
        self._label_fns[key] = fn

    def get(self, idx):
        event_id = self.event_ids[idx]
        event = self.batch_df.loc[event_id]
        event = pd.merge(event, self.sensor_df, on="sensor_id")
        if self.include_auxiliary == False:
            event.drop(event[event.auxiliary == 0.5].index)
        
        x_feats = self.x_features.copy()
        if 'scattering' in self.x_features:
            x_feats.remove('scattering')
        if 'absorption' in self.x_features:
            x_feats.remove('absorption')
        x = event[x_feats].values
        x = torch.tensor(x, dtype=torch.float32)
        data = Data(x=x, n_pulses=torch.tensor(x.shape[0], dtype=torch.int32), features=x_feats)

        # Add ice transparency data
        z = data.x[:, 2].numpy()
        if 'scattering' in self.x_features:
            scattering = torch.tensor(self.f_scattering(z), dtype=torch.float32).view(-1, 1)
            data.x = torch.cat([data.x, scattering], dim=1)
        if 'absorption' in self.x_features:
            absorption = torch.tensor(self.f_absorption(z), dtype=torch.float32).view(-1, 1)
            data.x = torch.cat([data.x, absorption], dim=1)

        # Downsample the large events
        if data.n_pulses > self.pulse_limit:
            data.x = data.x[np.random.choice(data.n_pulses, self.pulse_limit)]
            data.n_pulses = torch.tensor(self.pulse_limit, dtype=torch.int32)

        # Builds graph from the k-nearest neighbours.
        if self.construct_graph == True:
            data.edge_index = knn_graph(
                data.x[:, [0, 1, 2]],  # x, y, z
                k=8,
                batch=None,
                loop=False
            )
        if self.y is not None:
            y = self.y.loc[idx, :].values
            y = torch.tensor(y, dtype=torch.float32)
            data.y = y
            if self._label_fns:
                for key in self._label_fns:
                    data[key] = self._label_fns[key](data)
            
            '''
            data.azimuth = self.y.loc[idx][self.y_features].azimuth
            data.zenith = self.y.loc[idx][self.y_features].zenith
            dirs = self.get_dir_vector(data.azimuth, data.zenith)
            data.direction = torch.tensor(self.get_dir_vector(data.azimuth, data.zenith).values)
            torch.reshape(data.direction, (1,3))
            '''
            
        return data

In [7]:
BATCH_ID = 1
TRAIN_PATH = DATA_PATH + 'train/'
batch_meta = get_metadata_pd(BATCH_ID, write=False)
event_ids = list(batch_meta['event_id'])
#x_feats = ['x', 'y', 'z', 'time', "charge", "qe", "auxiliary", 'scattering', 'absorption']
#x_feats = ['x', 'y', 'z', 'time', "charge", "auxiliary"]
x_feats = ['x', 'y', 'z', 'time', "charge", "qe", "auxiliary", 'scattering']
y_feats = ['zenith', 'azimuth']
y = batch_meta[y_feats].reset_index(drop=True)
dir_x = np.cos(y.azimuth) * np.sin(y.zenith)
dir_y = np.sin(y.azimuth) * np.sin(y.zenith)
dir_z = np.cos(y.zenith)
directions = pd.concat({'direction_x':dir_x, 'direction_y':dir_y, 'direction_z':dir_z}, axis = 1)
directions.head()

,direction_x,direction_y,direction_z
0,0.271161,-0.826088,-0.494015
1,0.913804,0.405607,0.021108
2,0.268879,0.618078,-0.738704
3,0.623491,-0.291423,0.725488
4,0.640648,0.490794,0.590501


In [8]:
dataset = IceCubeDataset(event_ids, BATCH_ID, TRAIN_PATH, f_scattering, 
                         f_absorption, sensor_df, y, x_feats, y_feats)

In [9]:
dataset.add_label(Direction())

graphnet [MainProcess] INFO     2023-04-26 06:51:34 - Direction._configure_root_logger - Writing log to logs/graphnet_20230426-065134.log


In [10]:
dataset.get(0)['direction']

tensor([ 0.2712, -0.8261, -0.4940])

In [11]:
features = x_feats
truth = y_feats

config = {
        #"path": '/kaggle/working/batch_1.db',
        #"inference_database_path": '/kaggle/working/batch_51.db',
        #"pulsemap": 'pulse_table',
        #"truth_table": 'meta_table',
        'neighbours': 8,
        'graph_builder_columns' : [0, 1, 2], # x, y, z
        'global_pooling_schemes' : ["min", "max", "mean"],
        "features": features,
        #"truth": truth,
        "index_column": 'event_id',
        #"run_name_tag": 'my_example',
        "batch_size": 32,
        "num_workers": 2,
        "target": 'direction',
        "early_stopping_patience": 5,
        "fit": {
                "max_epochs": 10,
                "gpus": [0],
                "distribution_strategy": None,
                },
        #'train_selection': '/kaggle/working/train_selection_max_200_pulses.csv',
        #'validate_selection': '/kaggle/working/validate_selection_max_200_pulses.csv',
        #'test_selection': None,
        #'base_dir': 'training'
}

In [13]:
from abc import abstractmethod
from typing import Any, Optional, Union, List, Dict

import numpy as np
import scipy.special
import torch
from torch import Tensor
from torch import nn
from torch.nn.functional import (
    one_hot,
    cross_entropy,
    binary_cross_entropy,
    softplus,
)

from graphnet.utilities.config import save_model_config
from graphnet.models.model import Model
from graphnet.utilities.decorators import final

# overriding graphnet VonMisesFischer3dLoss and parent LossFunction
class vMF_Loss(Model):
    """Base class for loss functions in `graphnet`."""

    @save_model_config
    def __init__(self, **kwargs: Any) -> None:
        """Construct `LossFunction`, saving model config."""
        super().__init__(**kwargs)

    @final
    def forward(  # type: ignore[override]
        self,
        prediction: Tensor,
        target: Tensor,
        weights: Optional[Tensor] = None,
        return_elements: bool = False,
    ) -> Tensor:

        target = target.reshape(-1, 3)
        
        eps = 1e-8
        kappa = prediction[:, 3]      
        logC  = -kappa + torch.log( ( kappa+eps )/( 1-torch.exp(-2*kappa)+2*eps ) )
        p = kappa.unsqueeze(1) * prediction[:, [0, 1, 2]]
        return -( (target*p).sum(dim=1) + logC ).mean() 

In [14]:
from typing import Any, Callable, List, Optional, Sequence, Tuple, Union, Dict
from pytorch_lightning.callbacks import EarlyStopping
from torch.optim.adam import Adam
#from graphnet.data.constants import FEATURES, TRUTH
from graphnet.models.standard_model import StandardModel
#from graphnet.models.detector.icecube import IceCubeKaggle
from graphnet.models.gnn import DynEdge
from graphnet.models.graph_builders import KNNGraphBuilder
from graphnet.models.task.reconstruction import DirectionReconstructionWithKappa, ZenithReconstructionWithKappa, AzimuthReconstructionWithKappa
from graphnet.training.callbacks import ProgressBar, PiecewiseLinearLR
#from graphnet.training.loss_functions import VonMisesFisher3DLoss, VonMisesFisher2DLoss
#from graphnet.training.labels import Direction
from graphnet.training.utils import make_dataloader
from graphnet.utilities.logging import Logger
from pytorch_lightning import Trainer
import pandas as pd
from graphnet.models.detector.detector import Detector

logger = Logger()

#override graphnet class
class IceCubeKaggle(Detector):
    """`Detector` class for Kaggle Competition."""

    # Implementing abstract class attribute
    features = features

    def _forward(self, data: Data) -> Data:
        """Ingest data, build graph, and preprocess features.
        Args:
            data: Input graph data.
        Returns:
            Connected and preprocessed graph data.
        """
        # Check(s) --- no we want to have flexible feature inputs
        # Preprocessing was already done
        data_features = [features[0] for features in data.features]
        features = data_features

        return data

def build_model(config: Dict[str,Any], train_dataloader: Any) -> StandardModel:
    """Builds GNN from config"""
    # Building model
    detector = IceCubeKaggle(
        graph_builder=KNNGraphBuilder(nb_nearest_neighbours=config['neighbours'], 
                                     columns=config['graph_builder_columns']),
    )
    detector.features = config['features']
    gnn = DynEdge(
        nb_inputs=detector.nb_outputs,
        global_pooling_schemes=config['global_pooling_schemes'],
    )

   # if config["target"] == 'direction':
    task = DirectionReconstructionWithKappa(
            hidden_size=gnn.nb_outputs,
            target_labels=config["target"],
            #loss_function=VonMisesFisher3DLoss(),
            loss_function = vMF_Loss(),
        )
    prediction_columns = [config["target"] + "_x", 
                              config["target"] + "_y", 
                              config["target"] + "_z", 
                              config["target"] + "_kappa" ]
    additional_attributes = ['zenith', 'azimuth', 'event_id']

    model = StandardModel(
        detector=detector,
        gnn=gnn,
        tasks=[task],
        optimizer_class=Adam,
        optimizer_kwargs={"lr": 1e-03, "eps": 1e-03},
        scheduler_class=PiecewiseLinearLR,
        scheduler_kwargs={
            "milestones": [
                0,
                len(train_dataloader) / 2,
                len(train_dataloader) * config["fit"]["max_epochs"],
            ],
            "factors": [1e-02, 1, 1e-02],
        },
        scheduler_config={
            "interval": "step",
        },
    )
    model.prediction_columns = prediction_columns
    model.additional_attributes = additional_attributes
    
    return model

graphnet [MainProcess] WARNING  2023-04-26 06:51:36 - warning - `icecube` not available. Some functionality may be missing.


In [15]:
def training_step(config: Dict[str, Any], dataset_) -> StandardModel:
    """Builds and trains GNN according to config."""
    logger.info(f"features: {config['features']}")
    logger.info(f"truth: {config['target']}")
    
    #archive = os.path.join(config['base_dir'], "train_model_without_configs")
    #run_name = f"dynedge_{config['target']}_{config['run_name_tag']}"
    
    # do train-test split as:
    train_len = int(0.7*dataset_.len())
    train_loader = DataLoader(dataset_[:train_len], batch_size=config['batch_size'], shuffle=True, follow_batch=config['target']) # shuffle data every epoch
    val_loader = DataLoader(dataset_[train_len:], batch_size=config['batch_size'], shuffle=False, follow_batch=config['target'])
    
    model = build_model(config, train_loader)

    # Training model
    callbacks = [
        EarlyStopping(
            monitor="val_loss",
            patience=config["early_stopping_patience"],
        ),
        ProgressBar(),
    ]

    model.fit(
        train_loader,
        val_loader,
        callbacks=callbacks,
        **config["fit"],
    )
    return model

In [16]:
model = training_step(config=config)

TypeError: training_step() missing 1 required positional argument: 'dataset_'

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
def make_dataset_for_batch(BATCH_ID):
    TRAIN_PATH = DATA_PATH + 'train/'
    batch_meta = get_metadata_for_batch(BATCH_ID, write=False)
    event_ids = list(batch_meta['event_id'])
    #x_feats = ['x', 'y', 'z', 'time', "charge", "qe", "auxiliary", 'scattering', 'absorption']
    #x_feats = ['x', 'y', 'z', 'time', "charge", "auxiliary"]
    x_feats = ['x', 'y', 'z', 'time', "charge", "qe", "auxiliary", 'scattering']
    y_feats = ['zenith', 'azimuth']
    y = batch_meta[y_feats].reset_index(drop=True)
    return dataset

inference_dataset = make_dataset_for_batch(2)
# get first 10,000 samples only for batch 2 to do inference on
inf_loader = DataLoader(inference_dataset[:10000], batch_size=config['batch_size'], shuffle=False)
results = model.predict_as_dataframe(
        gpus = [0],
        dataloader = inf_loader,
        prediction_columns=model.prediction_columns,
        additional_attributes=model.additional_attributes,
    )